# Notebook that does the processing of foraging session trajectories

### Import functions necessary to do the processing 

In [1]:
import os
import gc
import glob
import time
from processing_TowerCoordinates import *
from processing_session_trajectory import *

### Make a list of the mice and sessions to process
Define the folder where your MOUXXX folders are, generate a list of MOUXXX folders and for each mice a list of sessions

In [2]:
# indicate where the data are
# path_to_data_folder is the path of the folder where you store your different mice.

# Windows:
# path_to_data_folder='C:'+os.sep+'Users'+os.sep+'MORVAN'+os.sep+'Documents'+os.sep+'patchouris'+os.sep+'Sample_Data'+os.sep
# Linux:
# path_to_data_folder '/home/david/Documents/Code/ForagingProject/Patchouris/patchouris/Sample_Data/'
# MacOs
#path_to_data_folder = '/Users/davidrobbe/Documents/Science/Data/ForagingMice/'


#path_to_data_folder='/home/david/Documents/David/Data/'
#path_to_data_folder='/LocalData/ForagingMice/JAK2Data'
#path_to_data_folder='/LocalData/ForagingMice/MaudData'
path_to_data_folder='/LocalData/ForagingMice/4TowersTaskMethodPaper_Data/AurelienData/'


pattern_of_MOU_Folders = os.path.join(path_to_data_folder, "MOU*")

# List all mice in the data folder (If you want to process all the mice in your data folder),
mice_list = [os.path.basename(path) for path in glob.glob(pattern_of_MOU_Folders)]
mice_list=sorted(mice_list)

# Print the number of mice, the list of mice, and add an empty line
print(f'Found {len(mice_list)} {"mice" if len(mice_list) > 1 else "mouse"} in the data folder: {", ".join(mice_list)}\n')


#If you want to process a subset of mice uncomment the line below and comment the 2 lines above

#mice_list: list[str] = ["MOU2334"]  # For processing  a single mice. Equivalent to mice_list = ["MOU2334"] but more correct as it forces to create a list of string
#mice_list=['MOU2329', 'MOU2330', 'MOU2331', 'MOU2332', 'MOU2333', 'MOU2334']
#print(f'Found {len(mice_list)} {"mice" if len(mice_list) > 1 else "mouse"} in the data folder:')

session_list = {}
for mouse in mice_list:
    mouse_folder = os.path.join(path_to_data_folder,mouse)
    session_list[mouse] = sorted([name for name in os.listdir(mouse_folder)
                           if os.path.isdir(os.path.join(mouse_folder, name))
                           and name.startswith('MOU')])
    nb_sessions = len(session_list[mouse])
    print(f'Hello, I\'m {mouse}! I have foraged for {nb_sessions} sessions:')
    print(session_list[mouse], '\n')








Found 22 mice in the data folder: MOUB6NN_11, MOUB6NN_13, MOUB6NN_15, MOUB6NN_2, MOUB6NN_4, MOUB6NN_6, MOUB6NN_9, MOUEml1_11, MOUEml1_12, MOUEml1_13, MOUEml1_15, MOUEml1_18, MOUEml1_20, MOUEml1_5, MOUEml1_8, MOURhoA_12, MOURhoA_14, MOURhoA_2, MOURhoA_5, MOURhoA_6, MOURhoA_8, MOURhoA_9

Hello, I'm MOUB6NN_11! I have foraged for 26 sessions:
['MOUB6NN_11_20241122-1051', 'MOUB6NN_11_20241122-1558', 'MOUB6NN_11_20241123-1143', 'MOUB6NN_11_20241123-1628', 'MOUB6NN_11_20241124-1146', 'MOUB6NN_11_20241124-1701', 'MOUB6NN_11_20241125-1041', 'MOUB6NN_11_20241125-1550', 'MOUB6NN_11_20241126-1042', 'MOUB6NN_11_20241126-1546', 'MOUB6NN_11_20241127-1105', 'MOUB6NN_11_20241127-1604', 'MOUB6NN_11_20241128-1115', 'MOUB6NN_11_20241128-1631', 'MOUB6NN_11_20241129-1127', 'MOUB6NN_11_20241129-1608', 'MOUB6NN_11_20241130-1135', 'MOUB6NN_11_20241130-1611', 'MOUB6NN_11_20241201-1127', 'MOUB6NN_11_20241201-1614', 'MOUB6NN_11_20241202-1105', 'MOUB6NN_11_20241202-1542', 'MOUB6NN_11_20241203-0748', 'MOUB6NN_11_2

## Process all the sessions in mice_list
### the key option here is wether to force processing or not (in case the data have already been processed)
#### for this the variable process should be set as true (to force) or false if the sessiobn has already been processed (if this is the case the name of the session has been saved in the  ListSessionsAnalyzed.txt file

In [3]:
# Add an overwrite flag
overwrite = False  # Set to True if you want to overwrite existing pickle files
bad_sessions = []

mice_to_process = mice_list

for mouse in mice_to_process:
    folder_path_mouse_to_process = os.path.join(path_to_data_folder, mouse)
    
    # Get the list of sessions
    sessions_to_process = sorted([name for name in os.listdir(folder_path_mouse_to_process)
                                  if os.path.isdir(os.path.join(folder_path_mouse_to_process, name))
                                  and name.startswith('MOU')])
    
    nb_sessions = len(sessions_to_process)
    print(f'Processing mouse {mouse}. There is/are {nb_sessions} sessions to process:')
    print(sessions_to_process, '\n')
    
    # Process each session
    for sessionindex,session_to_process in enumerate(sessions_to_process):
        print(f'Processing the trajectory of session {session_to_process}')
        if sessionindex==0: # for the first session of a given animal, we get the trapeze coordinates and then we will resuse them for the remaining sesions
            trapeze_width, towers_coordinates = get_trapeze_and_tower_data(folder_path_mouse_to_process, session_to_process)
            all_trapezes_coordinates_cm,towers_coordinates_cm= generate_trapeze_and_tower_coordinates(towers_coordinates, trapeze_width)
            
        
        # Define the pickle file path
        output_pickle_filename = f"{session_to_process}_basic_processing_output.pickle"
        output_pickle_filepath = os.path.join(folder_path_mouse_to_process, session_to_process, output_pickle_filename)
        
        # Check if the pickle file already exists
        if not overwrite and os.path.exists(output_pickle_filepath):
            print(f'Pickle file already exists for session {session_to_process}, skipping processing.')
            continue  # Skip processing if the file exists and overwrite is False
        
        # Run the processing if file doesn't exist or overwrite is True
        output=process_trajectory(folder_path_mouse_to_process, session_to_process,all_trapezes_coordinates_cm,towers_coordinates_cm)
        if output is not None:
            bad_sessions.append(output)
        print('#########################\n')

Processing mouse MOUB6NN_11. There is/are 26 sessions to process:
['MOUB6NN_11_20241122-1051', 'MOUB6NN_11_20241122-1558', 'MOUB6NN_11_20241123-1143', 'MOUB6NN_11_20241123-1628', 'MOUB6NN_11_20241124-1146', 'MOUB6NN_11_20241124-1701', 'MOUB6NN_11_20241125-1041', 'MOUB6NN_11_20241125-1550', 'MOUB6NN_11_20241126-1042', 'MOUB6NN_11_20241126-1546', 'MOUB6NN_11_20241127-1105', 'MOUB6NN_11_20241127-1604', 'MOUB6NN_11_20241128-1115', 'MOUB6NN_11_20241128-1631', 'MOUB6NN_11_20241129-1127', 'MOUB6NN_11_20241129-1608', 'MOUB6NN_11_20241130-1135', 'MOUB6NN_11_20241130-1611', 'MOUB6NN_11_20241201-1127', 'MOUB6NN_11_20241201-1614', 'MOUB6NN_11_20241202-1105', 'MOUB6NN_11_20241202-1542', 'MOUB6NN_11_20241203-0748', 'MOUB6NN_11_20241203-1644', 'MOUB6NN_11_20241204-1152', 'MOUB6NN_11_20241204-1640'] 

Processing the trajectory of session MOUB6NN_11_20241122-1051
Pickle file already exists for session MOUB6NN_11_20241122-1051, skipping processing.
Processing the trajectory of session MOUB6NN_11_2024112

## Verify that there is no bad sessions 

In [4]:
print(bad_sessions)

['MOURhoA_12_20240723-1120']
